# 言語処理100本ノック 2020 (Rev 2)


## 第5章: 係り受け解析

In [1]:
!cabocha -f1 ai.ja.txt >ai.ja.txt.parsed


### 40. 係り受け解析結果の読み込み（形態素）

In [13]:
# https://qiita.com/kazasiki/items/99505a5005aebe2efb7a
# https://taku910.github.io/cabocha/
# *行の出力形式
# 文節番号（0から始まる整数）
# 係り先の文節番号
# 文節に含まれる形態素の役割（という言い方でいいのか？？）。
#    例えば、0/1 の場合、主辞が 0 番目の形態素 、機能語が 一番目の形態素 と読むらしい
# 係り受けスコア。
#    大きな値ほど係りやすい　らしい
# 形態素解析
#   表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音

import re

class Morph():
  def __init__(self, surface, base, pos, pos1):
    self.surface = surface
    self.base = base
    self.pos = pos
    self.pos1 = pos1
  def __str__(self):
    return '"{}","{}","{}","{}"'.format(self.surface, self.base, self.pos, self.pos1)
  def __repr__(self):
    return str(self)

morphs = []
with open('ai.ja.txt.parsed') as f:
  for i, line in enumerate(f):
    if re.search(r"^EOS$",line):
      continue
    if re.search(r"^$",line):
      continue
    if re.search(r"^\*",line):
      # *行
      pass
    else:
      s = line.split('\t')
      if re.search(r"^$",s[0]):
        continue
      t = s[1].split(',')
      morphs.append(Morph(s[0], t[6], t[0], t[1]))

morphs

["人工","人工","名詞","一般",
 "知能","知能","名詞","一般",
 "人工","人工","名詞","一般",
 "知能","知能","名詞","一般",
 "（","（","記号","括弧開",
 "じん","じん","名詞","一般",
 "こうち","こうち","名詞","一般",
 "のう","のう","助詞","終助詞",
 "、","、","記号","読点",
 "、","、","記号","読点",
 "AI","*
 ","名詞","一般",
 "〈","〈","記号","括弧開",
 "エーアイ","*
 ","名詞","固有名詞",
 "〉","〉","記号","括弧閉",
 "）","）","記号","括弧閉",
 "と","と","助詞","格助詞",
 "は","は","助詞","係助詞",
 "、","、","記号","読点",
 "「","「","記号","括弧開",
 "『","『","記号","括弧開",
 "計算","計算","名詞","サ変接続",
 "（","（","記号","括弧開",
 "）","）","記号","括弧閉",
 "』","』","記号","括弧閉",
 "という","という","助詞","格助詞",
 "概念","概念","名詞","一般",
 "と","と","助詞","並立助詞",
 "『","『","記号","括弧開",
 "コンピュータ","コンピュータ","名詞","一般",
 "（","（","記号","括弧開",
 "）","）","記号","括弧閉",
 "』","』","記号","括弧閉",
 "という","という","助詞","格助詞",
 "道具","道具","名詞","一般",
 "を","を","助詞","格助詞",
 "用い","用いる","動詞","自立",
 "て","て","助詞","接続助詞",
 "『","『","記号","括弧開",
 "知能","知能","名詞","一般",
 "』","』","記号","括弧閉",
 "を","を","助詞","格助詞",
 "研究","研究","名詞","サ変接続",
 "する","する","動詞","自立",
 "計算","計算","名詞","サ変接続",
 "機","機","名詞","接尾",
 "科学","

In [37]:

# https://qiita.com/kazasiki/items/99505a5005aebe2efb7a
# https://taku910.github.io/cabocha/
# *行の出力形式
# 文節番号（0から始まる整数）
# 係り先の文節番号
# 文節に含まれる形態素の役割（という言い方でいいのか？？）。
#    例えば、0/1 の場合、主辞が 0 番目の形態素 、機能語が 一番目の形態素 と読むらしい
# 係り受けスコア。
#    大きな値ほど係りやすい　らしい
# 形態素解析
#   表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音

import re

class Morph():
  def __init__(self, surface, base, pos, pos1):
    self.surface = surface
    self.base = base
    self.pos = pos
    self.pos1 = pos1
  def __str__(self):
    return '<{},{},{},{}>'.format(self.surface, self.base, self.pos, self.pos1)
  def __repr__(self):
    return str(self)

class Chunk():
  def __init__(self, id, morphs, dst, srcs):
    self.id = id
    self.morphs = morphs
    self.dst = dst
    self.srcs = srcs
  def __str__(self):
    return '{}, {},{},{}'.format(self.id, self.morphs, self.dst, self.srcs)
  def __repr__(self):
    return str(self)



statements = []
chunks = []
c = None
with open('ai.ja.txt.parsed') as f:
  for i, line in enumerate(f):
    if re.search(r"^EOS$",line):
      if len(chunks) > 0:
        # 係り元文節INDEXのリスト作成
        for ch in chunks:
          if ch.dst < 0:
            continue
          chunks[ch.dst].srcs.append(ch.id)
        statements.append(chunks)
      c = None
      chunks = []
      continue
    if re.search(r"^$",line):
      continue
    if re.search(r"^\*",line):
      # 係り受け
      s = line.split(' ')
      c = Chunk(int(s[1]), [], int(s[2].replace("D","")), [])
      chunks.append(c)
    else:
      # 形態素
      s = line.split('\t')
      if re.search(r"^$",s[0]):
        continue
      t = s[1].split(',')
      c.morphs.append(Morph(s[0], t[6], t[0], t[1]))

statements

[[0, [<人工,人工,名詞,一般>, <知能,知能,名詞,一般>],-1,[]],
 [0, [<人工,人工,名詞,一般>, <知能,知能,名詞,一般>],17,[],
  1, [<（,（,記号,括弧開>, <じん,じん,名詞,一般>, <こうち,こうち,名詞,一般>, <のう,のう,助詞,終助詞>, <、,、,記号,読点>, <、,、,記号,読点>],17,[],
  2, [<AI,*
  ,名詞,一般>],3,[],
  3, [<〈,〈,記号,括弧開>, <エーアイ,*
  ,名詞,固有名詞>, <〉,〉,記号,括弧閉>, <）,）,記号,括弧閉>, <と,と,助詞,格助詞>, <は,は,助詞,係助詞>, <、,、,記号,読点>],17,[2],
  4, [<「,「,記号,括弧開>, <『,『,記号,括弧開>, <計算,計算,名詞,サ変接続>],5,[],
  5, [<（,（,記号,括弧開>, <）,）,記号,括弧閉>, <』,』,記号,括弧閉>, <という,という,助詞,格助詞>],9,[4],
  6, [<概念,概念,名詞,一般>, <と,と,助詞,並立助詞>],9,[],
  7, [<『,『,記号,括弧開>, <コンピュータ,コンピュータ,名詞,一般>],8,[],
  8, [<（,（,記号,括弧開>, <）,）,記号,括弧閉>, <』,』,記号,括弧閉>, <という,という,助詞,格助詞>],9,[7],
  9, [<道具,道具,名詞,一般>, <を,を,助詞,格助詞>],10,[5, 6, 8],
  10, [<用い,用いる,動詞,自立>, <て,て,助詞,接続助詞>],12,[9],
  11, [<『,『,記号,括弧開>, <知能,知能,名詞,一般>, <』,』,記号,括弧閉>, <を,を,助詞,格助詞>],12,[],
  12, [<研究,研究,名詞,サ変接続>, <する,する,動詞,自立>],13,[10, 11],
  13, [<計算,計算,名詞,サ変接続>, <機,機,名詞,接尾>, <科学,科学,名詞,一般>],14,[12],
  14, [<（,（,記号,括弧開>, <）,）,記号,括弧閉>, <の,の,助詞,連体化>],15,[13],
  15, [<一,一,名詞,数>, <分野,分野,名詞,一般>, 

In [23]:
" ".join(str([1,2,3]))

'[ 1 ,   2 ,   3 ]'